In [1]:
import molsysmt as msm

In [2]:
msm.get_form('pdb_id:1vii')

AttributeError: module 'molsysmt.form.file_trjpk' has no attribute 'is_form'

In [ ]:
msm.is_a_molecular_system('pdb_id:1vii')

In [ ]:
molsys = msm.convert('pdb_id:1vii', to_form='molsysmt.MolSys')

In [ ]:
msm.form._dict_convert

In [ ]:
msm.form.file_trjpk.form_name

In [ ]:
molsys = msm.convert('pdb_id:1vii', to_form='molsysmt.MolSys')
molsys = msm.basic.remove(molsys, selection='group_type==["water", "ion"]')
molsys = msm.basic.remove(molsys, selection='atom_type=="H"')
molsys = msm.build.add_missing_terminal_cappings(molsys, N_terminal='ACE', C_terminal='NME')
molsys = msm.build.add_missing_hydrogens(molsys, pH=7.4)

In [ ]:
aa = msm.convert([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}], to_form='molsysmt.MolecularMechanics')

In [ ]:
aa.forcefield

In [ ]:
msm.has_attribute([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}], 'kappa')

In [ ]:
msm.where_is_attribute([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}], 'kappa')

In [ ]:
molsys = msm.build.solvate([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                   box_shape='truncated octahedral', clearance='14.0 angstroms',
                   to_form='molsysmt.MolSys', engine="OpenMM")

In [ ]:
molsys.molecular_mechanics.flexible_constraints